In [1]:
import logging
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from utils.arg_extract import get_args
from utils.experiment_evaluator import evaluate_experiment
from utils.experiment import Experiment
from argparse import Namespace
from utils.experiment_evaluator import image_prepro, create_evaluation_dataloader, get_test_predictions_pairs, save_sequence_plots
import os
from utils.io import save
plt.ioff()
from utils.io import load
batch_images_all = load('batches.pickle')

data_directory = '../experiments_results/'
experiments = [dI for dI in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, dI))]

In [2]:
# dd = {'experiment_name': 'ar_lstm_batch_16_samples_10_in_5_out_20_normal_lr_0.0001_dataset_original_16h_patience_5',
#         'num_epochs': 1,
#         'num_workers': 1,
#         'test_starting_point': 15,
#         'num_total_output_frames':80,
#         'debug': False}

# args_new = Namespace(**dd)
# experiment = Experiment(args_new)
# experiment.load_from_disk(test=True)
    
    
# dataloaders = {"Test": experiment.dataloaders['test'],
#                "Lines": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Lines/'), experiment.args.normalizer_type),
#                "Double_Drop": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Double_Drop/'), experiment.args.normalizer_type),
#                "Illumination_135": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Illumination_135/'), experiment.args.normalizer_type),
#                "Shallow_Depth": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Shallow_Depth/'), experiment.args.normalizer_type),
#                "Smaller_Tub": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Smaller_Tub/'), experiment.args.normalizer_type),
#                "Bigger_Tub": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Bigger_Tub/'), experiment.args.normalizer_type),
#                 "Fixed_Tub": create_evaluation_dataloader(os.path.join(experiment.dirs['data_base'], 'Fixed_Tub_10/'), experiment.args.normalizer_type)

#                }

# batch_images_all = {}
# for dset, dloader in dataloaders.items():
#     it = iter(dloader)
#     for i in range(1,4):
#         batch_images_all[dset+'_%s'%i] = next(it)
        
# save(batch_images_all, 'batches.pickle')

In [3]:
def plot_all_tests_for_experiment(experiment, exp, belated):
    frames = [0, 9, 19, 29, 39, 59, 79]
    fig, ax = plt.subplots(int(len(batch_images_all)/3) * 2, len(frames), figsize = (19, 45))

    
    for i, k in enumerate(list(batch_images_all.keys())[::3]):
        batch_images = batch_images_all[k]
        output_frames, target_frames = get_test_predictions_pairs(experiment.model, belated, 
                                                              batch_images, 
                                                              args_new.test_starting_point-experiment.args.num_input_frames, 
                                                              args_new.num_total_output_frames)
        output_frames = image_prepro(output_frames, experiment.normalizer)
        target_frames = image_prepro(target_frames, experiment.normalizer)

        t = 0
        ax[2*i, t].set_ylabel(k.upper(), rotation=45, fontsize=20)
        ax[2*i, t].yaxis.set_label_coords(-0.2,0.5)
        ax[2*i, t].get_yaxis().set_ticks([])
        ax[2*i, t].spines['top'].set_visible(False)
        ax[2*i, t].spines['right'].set_visible(False)
        ax[2*i, t].spines['bottom'].set_visible(False)
        ax[2*i, t].spines['left'].set_visible(False)
        ax[2*i, t].tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False)

        ax[2*i+1, t].set_ylabel('Prediction', rotation=45, fontsize=20)
        ax[2*i+1, t].yaxis.set_label_coords(-0.2,0.5)
        ax[2*i+1, t].get_yaxis().set_ticks([])
        ax[2*i+1, t].spines['top'].set_visible(False)
        ax[2*i+1, t].spines['right'].set_visible(False)
        ax[2*i+1, t].spines['bottom'].set_visible(False)
        ax[2*i+1, t].spines['left'].set_visible(False)
        ax[2*i+1, t].tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False)

        for t in range(len(frames)):
            if t > 0:
                ax[2*i, t].axis('off')
                ax[2*i+1, t].axis('off')
            if i == 0:
                ax[2*i, t].set_title('t = ' + str(frames[t]+1), fontsize=30)
            ax[2*i, t].imshow(target_frames[0, frames[t], :, :], cmap='gray', vmin=0, vmax=1)
            ax[2*i+1, t].imshow(output_frames[0, frames[t], :, :], cmap='gray', vmin=0, vmax=1)
    #         ax[i,0].set_ylabel('test', rotation = 0)

    exp_name = exp + '_belated' if belated else exp
    fig.suptitle(exp_name, fontsize=24) # or plt.suptitle('Main title')
    fig.tight_layout()
    plt.subplots_adjust(hspace=0.00, top=0.95, wspace=0.05)
#     plt.show()
    plt.savefig('qualitative/Individual_Models/%s.png'% exp_name, format='png')
    plt.close()

In [4]:
# [e for e in experiments if 'ar_lstm' in e and 'dilated' not in e]

In [5]:
# experiments.sort()
# for exp in experiments:
#     print(exp)
#     dd = {'experiment_name': exp,
#             'num_epochs': 1,
#             'num_workers': 1,
#             'test_starting_point': 15,
#             'num_total_output_frames':80,
#             'debug': False}

#     args_new = Namespace(**dd)
#     experiment = Experiment(args_new)
#     experiment.load_from_disk(test=True)
    
#     plot_all_tests_for_experiment(experiment, exp, belated=False)
#     if experiment.args.model_type == 'convlstm' or experiment.args.model_type == 'predrnn':
#         plot_all_tests_for_experiment(experiment, exp, belated=True)

In [6]:
def get_experiment_predictions(exp, batch_images):

    if 'belated' in exp:
        exp = exp[:-8]
        belated = True
    else:
        belated = False

#     print(exp)
    dd = {'experiment_name': exp,
            'num_epochs': 1,
            'num_workers': 1,
            'test_starting_point': 15,
            'num_total_output_frames':80,
            'debug': False}

    args_new = Namespace(**dd)
    experiment = Experiment(args_new)
    experiment.load_from_disk(test=True)
    
    output_frames, target_frames = get_test_predictions_pairs(experiment.model, belated, 
                                                              batch_images, 
                                                              args_new.test_starting_point-experiment.args.num_input_frames, 
                                                              args_new.num_total_output_frames)
    return image_prepro(output_frames, experiment.normalizer), image_prepro(target_frames, experiment.normalizer)

In [7]:
real_exp_name = {'ar_lstm_': 'LSTM',
                 'convlstm': 'ConvLSTM',
                 'predrnn_': 'PredRNN++',
                 'resnet_b': 'ResNet',
                 'resnet_d': 'ResNet Dil',
                 'unet_bat': 'U-Net'
                }

In [8]:
# ar_lstm_batch_16_samples_10_in_5_out_20_normal_lr_0.0001_dataset_original_16h_patience_5
# convlstm_batch_8_samples_5_in_5_out_10_normal_lr_0.001_16h_c_belated
# predrnn_batch_4_samples_5_in_5_out_20_normal_lr_0.0001_dataset_original_24h_patience_3_belated
# resnet_batch_16_samples_5_in_5_out_10_normal_lr_0.001
# unet_batch_16_samples_5_in_5_out_20_normal_lr_0.0001_dataset_original_16h_patience_7

exp_to_plot = [
 'ar_lstm_batch_16_samples_10_in_5_out_20_normal_lr_0.0001_dataset_original_16h_patience_5',
 'convlstm_batch_8_samples_5_in_5_out_10_normal_lr_0.001_16h_c_belated',
 'predrnn_batch_4_samples_5_in_5_out_20_normal_lr_0.0001_dataset_original_24h_patience_3_belated',
 'resnet_batch_16_samples_5_in_5_out_10_normal_lr_0.001',
'resnet_dilated_batch_16_samples_5_in_5_out_10_normal_lr_0.001_dataset_original_16h_patience_7',
 'unet_batch_16_samples_5_in_5_out_20_normal_lr_0.0001_dataset_original_16h_patience_7']

In [11]:
def plot_model_comparison(test_set):
    batch_images = batch_images_all[test_set]
    frames = [9, 19, 29, 39, 59, 79]
    fig, ax = plt.subplots(len(exp_to_plot)+1, len(frames), figsize = (13.5, 17))

    fontsize = 16

    for i, exp in enumerate(exp_to_plot):
        output_frames, target_frames = get_experiment_predictions(exp, batch_images)
#         print(output_frames.min(), output_frames.max())
        i += 1
        ax[i, 0].set_ylabel(real_exp_name[exp[:8]], rotation=45, fontsize=fontsize)
        for t in range(len(frames)):
            ax[i, t].imshow(output_frames[0, frames[t], :, :], cmap='gray', vmin=0, vmax=1)
            ax[i, t].yaxis.set_label_coords(-0.2,0.5)
            ax[i, t].get_yaxis().set_ticks([])
            ax[i, t].spines['top'].set_visible(False)
            ax[i, t].spines['right'].set_visible(False)
            ax[i, t].spines['bottom'].set_visible(False)
            ax[i, t].spines['left'].set_visible(False)
            ax[i, t].tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False)
            
    i = 0
    t = 0
    ax[i, t].set_ylabel('Ground truth', rotation=45, fontsize=fontsize, labelpad=100)
    for t in range(len(frames)):
        ax[i, t].imshow(target_frames[0, frames[t], :, :], cmap='gray', vmin=0 , vmax=1)
        ax[i, t].yaxis.set_label_coords(-0.2,0.5)
        ax[i, t].get_yaxis().set_ticks([])
        ax[i, t].spines['top'].set_visible(False)
        ax[i, t].spines['right'].set_visible(False)
        ax[i, t].spines['bottom'].set_visible(False)
        ax[i, t].spines['left'].set_visible(False)
        ax[i, t].tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False)
        ax[i, t].set_title('t = ' + str(frames[t]+1), fontsize=fontsize)

    fig.suptitle(test_set, fontsize=fontsize*1.5) # or plt.suptitle('Main title')
    fig.tight_layout()
    plt.subplots_adjust(hspace=0.01, top=0.90, wspace=0.03)
    plt.savefig('qualitative/Best_Model_Comparison/Evaluate_%s.png'% test_set, format='png')
    plt.savefig('qualitative/Best_Model_Comparison/PDF_Evaluate_%s.pdf'% test_set, format='pdf')
#     plt.show()
    plt.close()

In [12]:
for test_set in batch_images_all.keys():
    print(test_set)
    plot_model_comparison(test_set)

Test_1
Test_2
Test_3
Lines_1
Lines_2
Lines_3
Double_Drop_1
Double_Drop_2
Double_Drop_3
Illumination_135_1
Illumination_135_2
Illumination_135_3
Shallow_Depth_1
Shallow_Depth_2
Shallow_Depth_3
Smaller_Tub_1
Smaller_Tub_2
Smaller_Tub_3
Bigger_Tub_1
Bigger_Tub_2
Bigger_Tub_3
Fixed_Tub_1
Fixed_Tub_2
Fixed_Tub_3


In [ ]:
output_frames, target_frames = get_experiment_predictions(exp, batch_images)